In [ ]:
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, TimeDistributed
import librosa
import librosa.display
import matplotlib.pyplot as plot
import IPython.display as ipd
import pylab
import pandas as pd
import numpy as np


In [ ]:
filename = "../data/tracks/unmastered/pianosolo_unmastered_001.wav"
filename_mastered = "../data/tracks/mastered/pianosolo_mastered_001_var1.wav"
y, sr = librosa.load(filename, sr=44100, mono=False)
y.shape

In [ ]:
y = librosa.to_mono(y)
print("sr:", sr)
y.shape



In [ ]:
ipd.Audio(filename)

In [ ]:
ipd.Audio(filename_mastered)

In [ ]:
hop_length = 512
n_fft = 2048


stftSource = librosa.stft(y, n_fft=n_fft, hop_length=hop_length).T
stftTarget = librosa.stft(ym, n_fft=n_fft, hop_length=hop_length).T

stftSource.shape




In [ ]:
unmasteredDf = pd.DataFrame(data=stftSource, index=np.array(range(0,2611)), columns=np.array(range(0,1025)))
masteredDf = pd.DataFrame(data=stftTarget, index=np.array(range(0,2611)), columns=np.array(range(0,1025)))

In [ ]:
x_train = unmasteredDf.loc[:2611,].values
x_test = unmasteredDf.loc[:2611,].values
y_train = masteredDf.loc[:2611,].values
y_test = masteredDf.loc[:2611,].values

In [ ]:
print(x_train)

In [ ]:


x_train = x_train.view(dtype=np.float32)[:, None]
x_test = x_test.view(dtype=np.float32)[:, None]
y_train = y_train.view(dtype=np.float32)[:, None]
y_test = y_test.view(dtype=np.float32)[:, None]

x_train.shape


In [ ]:
x_train

In [ ]:
import keras.backend as K
from keras.callbacks import EarlyStopping

In [ ]:
K.clear_session()
model = Sequential()
model.add(LSTM(2050, return_sequences=True, input_shape=(1,2050)))
model.add(Dense(2050))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=4, verbose=1)

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=1, verbose=1, callbacks=[early_stop])

In [ ]:
#save model 
model_json = model.to_json() 
with open('../data/model_LSTM.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('../data/model_LSTM.h5')

In [ ]:
#load model
from keras.models import model_from_json
K.clear_session()
json_file = open('../data/model_LSTM.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("../data/model_LSTM.h5")
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
loaded_model.summary()

In [ ]:
loaded_model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=1, callbacks=[early_stop])

In [ ]:
print(y_train)

In [ ]:
y_pred = loaded_model.predict(x_test)
y_pred = np.squeeze(y_pred, axis=1)
print(y_pred)

In [ ]:
y_pred.shape

In [ ]:
y_pred_complex =  []
type(y_pred_complex)

In [ ]:
for x in range(0, 1307, 1):
    for y in range(0,2050,2):
        y_pred_complex.append(np.complex(y_pred[x,y], y_pred[x,y+1]))
        
print(y_pred_complex)

In [ ]:
y_pred_complex = np.array(y_pred_complex)
y_pred_complex = y_pred_complex.reshape(1307, 1025)

In [ ]:
x = librosa.istft(y_pred_complex.T, hop_length=hop_length)
filename_output='../data/output/pianosolo_mastered_001.wav'
librosa.output.write_wav(filename_output, x, sr)

In [ ]:
ipd.Audio(filename_output)